In [19]:
import sys
sys.path.append('..')
sys.path.append('../..')

import pandas as pd


from src.tools import *
from src.database import Database




seleziono le transazioni che mi interessa contabilizzare, scegliere momento iniziale e finale

In [ ]:
# scegliere start e stop
txs = Database('db.db').txs(start='2017-01-01', stop='2023-01-01').copy()

Ricerca delle transazioni contabilizzate due volte, cioè transazioni in uscita da un recipient ed in entrata in un altro (l'IN dell'una è uguale all'OUT dell'altra e viceversa)

inserisco una piccola percentuale di tolleranza per includere anche gli errori di calcolo degli exchanges (come commissioni di transazioni non contabilizzate o quant'altro).

a queste transazioni vengono assegnati 3 labels:
1. label1: 'internal'
2. label2: il recipient in cui la tx arriva o da cui proviene
3. label3 l'id della transazione speculare

In [20]:
txs[['label1', 'label2', 'label3']] = [None, None, None]
def label(id0, id1):
    txs.loc[id0, 'label1']='internal'
    txs.loc[id1, 'label1']='internal'
    txs.loc[id0, 'label2']= id1[1]
    txs.loc[id1, 'label2']= id0[1]
    txs.loc[id0, 'label3']= id1[0]
    txs.loc[id1, 'label3']= id0[0]


def sim(a,b, tolerance_percent):
    if b == 0: return False
    if a == 0: return False
    return True if abs(a/b -1) <= tolerance_percent else False

 flagghiamo le tx con id uguale (da electrum a electrum o da eth a eth)

In [21]:

ug = txs[txs['type']=='ext'].groupby(level='id')
for name, data in ug:
    if (len(data)==2):
            ids = tuple(data.index)
            label(ids[0], ids[1])


la funzione search restituisce un elenco di coppie (tuple) contenenti gli index delle transazioni da contrassegnare.
E stampa quelle che in cui la IN avviene prima dell'OUT, dovuto probabilmente a problemi di fuso orario o a scelte di contabilizzazione dei diversi recipients. Controllarle una per una.

In [22]:
def search(tolerance_time, tolerance_amount, df):
    ext = df[(df['type']=='ext' )| (df['type']=='other')]
    ext=ext[ext['label1']!='internal']
    
    regular = 0
    inversion = 0

    ids = []
    couples = []

    for i, r in ext.iterrows():
        t = r['timestamp']
        mask = (ext['timestamp']>=t ) & (ext['timestamp']<(t+tolerance_time))
        EXT = ext[mask]
    

        for ii, rr in EXT.iterrows():
            if i != ii:
                if sim(r['in'], rr['out'], tolerance_amount):
                   
                    inversion += 1
                                      
                    ids.append(i)
                    ids.append(ii)
                    couples.append((i, ii))
                    
                elif sim(r['out'], rr['in'], tolerance_amount):
                 
                    regular += 1
                        
                    ids.append(i)
                    ids.append(ii)
                    couples.append((i, ii))
                
    print('REGULAR:',regular, 'INVERSION', inversion)
    return couples       


cerchiamo le tx con in e out esattamente uguali (a meno di 1/1000) e flagghiamole

In [23]:
# tempo tra le transazioni, 5 ore di separazione permesse, perchè spesso i recipients non forniscono
# i dati sulla timezone utilizzata.

tolerance_time = 5 *60*60*1000
tolerance_amount = 0.001
ids = search(tolerance_time, tolerance_amount, txs)

REGULAR: 0 INVERSION 0


In [24]:
for id in ids:
    rec1 = id[0][1]
    rec2 = id[1][1]
    if (rec1 != rec2): label(id[0], id[1])
    else: print(id)

### cerchiamo con un po' di tolleranza in più e controlliamo a mano

In [25]:
tolerance_time = 5 *60*60*1000 #tempo tra le transazioni, 5 ore di separazione allowed
tolerance_amount = 0.01
df = txs[txs['label1']!='internal']
ids = search(tolerance_time, tolerance_amount, df)

REGULAR: 0 INVERSION 1


In [26]:
def show(ids, i):
    cip = txs[(txs.index==ids[i][0]) | (txs.index==ids[i][1])].copy()
    return date(cip)
def flagi(ids,i):
    label(ids[i][0], ids[i][1])

In [27]:
show(ids,0)

,,date,timestamp,type,info,in,in_iso,out,out_iso,fee,fee_iso,label1,label2,label3
id,recipients,,,,,,,,,,,,,
34923105,therocktrading,2019-05-24 07:55:02 UTC,1558684502000,ext,None,201.5,EUR,NaN,None,NaN,None,None,None,None
33687191,therocktrading,2019-05-24 07:56:48 UTC,1558684608000,ext,IT36Q0200816909000004696350,NaN,None,200.0,EUR,NaN,None,None,None,None


flaggiamole singolarmente

In [28]:
#flagi(ids,0)

### salviamo le transazioni flaggate in un nuovo database.


In [29]:
DB = Database('db1.db')

In [30]:
DB.save('txs', txs)